In [0]:
%sql
-- Revenue with 7-day moving average
WITH daily AS (
  SELECT event_date, SUM(revenue) as rev
  FROM gold.products GROUP BY event_date
)
SELECT event_date, rev,
  AVG(rev) OVER (ORDER BY event_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as ma7
FROM daily;

-- Conversion funnel
SELECT category_code,
  SUM(views) as views,
  SUM(purchases) as purchases,
  ROUND(SUM(purchases)*100.0/SUM(views), 2) as conversion_rate
FROM gold.products
GROUP BY category_code;

-- Customer tiers
SELECT
  CASE WHEN cnt >= 10 THEN 'VIP'
       WHEN cnt >= 5 THEN 'Loyal'
       ELSE 'Regular' END as tier,
  COUNT(*) as customers,
  AVG(total_spent) as avg_ltv
FROM (SELECT user_id, COUNT(*) cnt, SUM(price) total_spent
      FROM silver.events WHERE event_type='purchase' GROUP BY user_id)
GROUP BY tier;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5143400626236408>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "-- Revenue with 7-day moving average\nWITH daily AS (\n  SELECT event_date, SUM(revenue) as rev\n  FROM gold.products GROUP BY event_date\n)\nSELECT event_date, rev,\n  AVG(rev) OVER (ORDER BY event_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as ma7\nFROM daily;\n\n-- Conversion funnel\nSELECT category_code,\n  SUM(views) as views,\n  SUM(purchases) as purchases,\n  ROUND(SUM(purchases)*100.0/SUM(views), 2) as conversion_rate\nFROM gold.products\nGROUP BY category_code;\n\n-- Customer tiers\nSELECT\n  CASE WHEN cnt >= 10 THEN 'VIP'\n       WHEN cnt >= 5 THEN 'Loyal'\n       ELSE 'Regular' END as tier,\n  COUNT(*) as customers,\n  AVG(total_spent) as avg_ltv\nFROM (SELECT user_id, COUNT(*) cnt, SUM(price) total_spent\n      FROM silver.events WHER

In [0]:
%sql
SELECT current_catalog(), current_schema();


In [0]:
%sql
SELECT current_catalog(), current_schema();


current_catalog(),current_schema()
ecommerce,gold


In [0]:
%sql
SELECT * FROM products;


product_id,product_name,revenue,purchases,conversion_rate


In [0]:
%sql
CREATE OR REPLACE TABLE products AS
SELECT
  product_id,
  product_name,
  SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS revenue,
  COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,
  ROUND(
    COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) * 100.0 /
    NULLIF(COUNT(CASE WHEN event_type = 'view' THEN 1 END), 0),
    2
  ) AS conversion_rate
FROM silver.events
GROUP BY product_id, product_name;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5143400626236414>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "CREATE OR REPLACE TABLE products AS\nSELECT\n  product_id,\n  product_name,\n  SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS revenue,\n  COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,\n  ROUND(\n    COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) * 100.0 /\n    NULLIF(COUNT(CASE WHEN event_type = 'view' THEN 1 END), 0),\n    2\n  ) AS conversion_rate\nFROM silver.events\nGROUP BY product_id, product_name;\n")

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents t

In [0]:
%sql
DESCRIBE TABLE silver.events;


col_name,data_type,comment
event_id,string,null
user_id,string,null
event_time,timestamp,null


In [0]:
%sql
CREATE OR REPLACE TABLE silver.events_enriched AS
SELECT
  event_id,
  user_id,
  event_time,
  DATE(event_time) AS event_date,

  -- simulated business fields
  CASE
    WHEN rand() < 0.7 THEN 'view'
    ELSE 'purchase'
  END AS event_type,

  CASE
    WHEN rand() < 0.5 THEN 'electronics'
    WHEN rand() < 0.8 THEN 'fashion'
    ELSE 'groceries'
  END AS category_code,

  CASE
    WHEN rand() < 0.7 THEN round(rand()*500 + 100, 2)
    ELSE 0
  END AS price
FROM silver.events;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE gold.products AS
SELECT
  category_code,
  SUM(CASE WHEN event_type='purchase' THEN price ELSE 0 END) AS revenue,
  COUNT(CASE WHEN event_type='purchase' THEN 1 END) AS purchases,
  ROUND(
    COUNT(CASE WHEN event_type='purchase' THEN 1 END) * 100.0 /
    NULLIF(COUNT(CASE WHEN event_type='view' THEN 1 END), 0),
    2
  ) AS conversion_rate
FROM silver.events_enriched
GROUP BY category_code;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE gold.daily_revenue AS
SELECT
  event_date,
  SUM(price) AS revenue
FROM silver.events_enriched
WHERE event_type='purchase'
GROUP BY event_date;


num_affected_rows,num_inserted_rows
